In [262]:
import numpy as np
import pandas as pd

In [263]:
school_df = pd.read_csv('Course Data/UIUC_Programs_with_Descs.csv', index_col=0)
school_df['CIP'] = np.nan

school_df

,Institution,Program,Course_Descriptions_Concat,CIP
0,UIUC,"Accountancy, BS",\nDevelops a foundation for understanding and ...,NaN
1,UIUC,"Accountancy + Data Science, BS",\nAddresses the conceptual framework and finan...,NaN
2,UIUC,"Actuarial Science, BSLAS",\nDevelops a foundation for understanding and ...,NaN
3,UIUC,"African American Studies, BALAS",\nThis course prepares accounting students to ...,NaN
4,UIUC,"Agricultural Leadership, Education, & Communic...",\nStakeholders' needs for reliable and relevan...,NaN
...,...,...,...,...
131,UIUC,"Strategic Brand Communications, MS",\nWill examine state and local tax laws preval...,NaN
132,UIUC,"Sustainable Urban Design, MSUD",\nPractice-intensive grammar review and vocabu...,NaN
133,UIUC,"Sustainable Urban Management, MS",\nIntroductory course focusing on listening co...,NaN
134,UIUC,"Taxation, MS",\nBasic composition course; problems of writte...,NaN


In [264]:
cip_codes = pd.read_excel('Course Data/UIUC_Courses.xlsx')
cip_codes.head()

,FICE,CIP,Level,Degree,Program Name,Region #,Region Name,Status
0,001775,01.0000,5,Bachelors,B.S. in Sustainability in Food & Environmental...,7,Prairie,Active
1,001775,01.0801,5,Bachelors,"B.S. in Agricultural Leadership, Education, an...",7,Prairie,Active
2,001775,01.0801,7,Masters,NaN,7,Prairie,Active
3,001775,01.0801,7,Masters,"M.S. in Agricultural Leadership, Education, an...",7,Prairie,Active
4,001775,01.0901,5,Bachelors,B.S. in Animal Sciences,7,Prairie,Active


In [265]:
def reformat(row):
    program_string = row[4]
    try:
        string_split = program_string.split(' in ')
        level = string_split[0].replace('.', '')
        dept = string_split[1]
        return [dept, level, str(dept + ', ' + level)]
    except:
        #print(program_string)
        return 'ERROR'

cip_codes[['dept', 'level', 'reformatted_program']] = cip_codes.apply(reformat, axis=1, result_type = 'expand')
cip_codes = pd.DataFrame(cip_codes)
cip_codes

,FICE,CIP,Level,Degree,Program Name,Region #,Region Name,Status,dept,level,reformatted_program
0,001775,01.0000,5,Bachelors,B.S. in Sustainability in Food & Environmental...,7,Prairie,Active,Sustainability,BS,"Sustainability, BS"
1,001775,01.0801,5,Bachelors,"B.S. in Agricultural Leadership, Education, an...",7,Prairie,Active,"Agricultural Leadership, Education, and Commun...",BS,"Agricultural Leadership, Education, and Commun..."
2,001775,01.0801,7,Masters,NaN,7,Prairie,Active,ERROR,ERROR,ERROR
3,001775,01.0801,7,Masters,"M.S. in Agricultural Leadership, Education, an...",7,Prairie,Active,"Agricultural Leadership, Education, and Commun...",MS,"Agricultural Leadership, Education, and Commun..."
4,001775,01.0901,5,Bachelors,B.S. in Animal Sciences,7,Prairie,Active,Animal Sciences,BS,"Animal Sciences, BS"
...,...,...,...,...,...,...,...,...,...,...,...
492,001775,90.5102,59,Center,Speech and Hearing Clinic,7,Prairie,Active,ERROR,ERROR,ERROR
493,001775,90.5201,59,Center,Center for Business and Public Policy,7,Prairie,Active,ERROR,ERROR,ERROR
494,001775,90.5202,59,Center,Executive Development Center,7,Prairie,Active,ERROR,ERROR,ERROR
495,001775,90.5206,59,Center,Bureau of Economic and Business Research,7,Prairie,Active,ERROR,ERROR,ERROR


In [284]:
count = 0
pd.set_option('display.max_colwidth', 50)
for i, program in enumerate(school_df['Program']):
    
    if count % 1 == 0:
        break
    
    if pd.notnull(school_df.iloc[i]['CIP']):
        continue
    program = program.replace('+', 'plus')
    print(program)
    possible_prog = cip_codes[cip_codes['reformatted_program'].str.contains(program) == True]
    unique_cip_codes = possible_prog['CIP'].unique()
    if len(unique_cip_codes) == 1:
        short_cip = str(unique_cip_codes[0][:6])
        school_df.CIP[i] = str(short_cip)
        count += 1
    elif len(unique_cip_codes) == 0:
        program_split = program.split()
        all_possible = pd.DataFrame(columns=cip_codes.columns)
        for keyword in program_split:
            if len(keyword) <= 2:
                continue
            clean_keyword = keyword.strip(',:')
            #print(clean_keyword)
            possible_prog = cip_codes[cip_codes['reformatted_program'].str.contains(clean_keyword) == True]
            #print(possible_prog)
            all_possible = pd.concat([all_possible, possible_prog], ignore_index=True).drop_duplicates('CIP')
        print(all_possible[['Program Name', 'CIP']].sort_values('Program Name'))
        cip = input('What CIP code to input?')
        if cip == 'na':
            count += 1
            continue
        school_df.CIP[i] = str(cip)
        count += 1
    else:
        print('MORE THAN ONE INITIAL MATCH')
        print(possible_prog[['Program Name', 'CIP']].sort_values('Program Name'))
        cip = input('What CIP code to input?')
        if cip == 'na':
            continue
        school_df.CIP[i] = str(cip)
        count += 1
        
            
print(count)
school_df.to_csv('All_School_Programs.csv')

0


In [283]:
pd.set_option('display.max_colwidth', 50)
school_df['CIP'].apply(str)
school_df.head(25)

,Institution,Program,Course_Descriptions_Concat,CIP
0,UIUC,"Accountancy, BS",\nDevelops a foundation for understanding and ...,52.03
1,UIUC,"Accountancy + Data Science, BS",\nAddresses the conceptual framework and finan...,30.70
2,UIUC,"Actuarial Science, BSLAS",\nDevelops a foundation for understanding and ...,52.13
3,UIUC,"African American Studies, BALAS",\nThis course prepares accounting students to ...,5.02
4,UIUC,"Agricultural Leadership, Education, & Communic...",\nStakeholders' needs for reliable and relevan...,1.08
5,UIUC,"Agricultural Leadership, Education, & Communic...",\nFirst course in calculus and analytic geomet...,1.08
6,UIUC,Agricultural Leadership & Science Education: A...,\nIntroduction for grad students to the centra...,1.08
7,UIUC,Agricultural Leadership & Science Education: A...,\nInterdisciplinary introduction to the basic ...,1.08
8,UIUC,"Anthropology, BALAS",\nFor students who have some prior knowledge o...,45.02
9,UIUC,"Anthropology: Archaeology, BALAS",\nFor students who have some prior knowledge o...,45.02


In [261]:
school_df.CIP[3]

5.02